## Python 代码优化技巧（二）

### 避免全局变量

**在函数中编写代码而不要将其写为全局变量。** 由于全局变量和局部变量实现方式不同，定义在全局范围内的代码运行速度会比定义在函数中的慢不少。通过将脚本语句放入到函数中，通常可带来 15% - 30% 的速度提升。

In [1]:
# 不推荐写法
import math


size = 10000
for x in range(size):
    for y in range(size):
        z = math.sqrt(x) + math.sqrt(y)

In [2]:
# 推荐写法
import math

def main():  # 定义到函数中，以减少全部变量使用
    size = 10000
    for x in range(size):
        for y in range(size):
            z = math.sqrt(x) + math.sqrt(y)

main()

### 避免 `.`

#### 避免模块和函数属性访问
每次使用`.`（属性访问操作符时）会触发特定的方法，如`__getattribute__()`和`__getattr__()`，这些方法会进行字典操作，因此会带来额外的时间开销。通过`from import`语句，可以消除属性访问。

In [ ]:
# 不推荐写法
import math

def computeSqrt(size: int):
    result = []
    for i in range(size):
        result.append(math.sqrt(i))
    return result

def main():
    size = 10000
    for _ in range(size):
        result = computeSqrt(size)

main()

In [ ]:
# 推荐写法，第一次优化 . 模块属性访问
from math import sqrt

def computeSqrt(size: int):
    result = []
    for i in range(size):
        result.append(sqrt(i))  # 避免math.sqrt的使用
    return result

def main():
    size = 10000
    for _ in range(size):
        result = computeSqrt(size)

main()

In [ ]:
# 推荐写法，第二次优化局部变量 sqrt，局部变量访问比全局变量快。
import math

def computeSqrt(size: int):
    result = []
    sqrt = math.sqrt  # 赋值给局部变量
    for i in range(size):
        result.append(sqrt(i))  # 避免math.sqrt的使用
    return result

def main():
    size = 10000
    for _ in range(size):
        result = computeSqrt(size)

main()

In [ ]:
# 推荐写法， 第三次优化函数属性 list.append() 方法
import math

def computeSqrt(size: int):
    result = []
    append = result.append
    sqrt = math.sqrt    # 赋值给局部变量
    for i in range(size):
        append(sqrt(i))  # 避免 result.append 和 math.sqrt 的使用
    return result

def main():
    size = 10000
    for _ in range(size):
        result = computeSqrt(size)

main()

#### 避免访问类内属性
避免 `.` 的原则也适用于类内属性，访问`self._value`的速度会比访问一个局部变量更慢一些。通过将需要频繁访问的类内属性赋值给一个局部变量，可以提升代码运行速度。

In [3]:
# 不推荐写法
import math
from typing import List

class DemoClass:
    def __init__(self, value: int):
        self._value = value
    
    def computeSqrt(self, size: int) -> List[float]:
        result = []
        append = result.append
        sqrt = math.sqrt
        for _ in range(size):
            append(sqrt(self._value))
        return result

def main():
    size = 10000
    for _ in range(size):
        demo_instance = DemoClass(size)
        result = demo_instance.computeSqrt(size)

main()

In [ ]:
# 推荐写法，将self._value 赋值给局部变量
import math
from typing import List

class DemoClass:
    def __init__(self, value: int):
        self._value = value
    
    def computeSqrt(self, size: int) -> List[float]:
        result = []
        append = result.append
        sqrt = math.sqrt
        value = self._value
        for _ in range(size):
            append(sqrt(value))  # 避免 self._value 的使用
        return result

def main():
    size = 10000
    for _ in range(size):
        demo_instance = DemoClass(size)
        demo_instance.computeSqrt(size)

main()

### 避免不必要的抽象
任何时候当你使用额外的处理层（比如装饰器、属性访问、描述器）去包装代码时，都会让代码变慢。大部分情况下，需要重新进行审视使用属性访问器的定义是否有必要，使用`getter/setter`函数对属性进行访问通常是 C/C++ 程序员遗留下来的代码风格。如果真的没有必要，就使用简单属性。

In [ ]:
# 不推荐写法。
class DemoClass:
    def __init__(self, value: int):
        self.value = value

    @property
    def value(self) -> int:
        return self._value

    @value.setter
    def value(self, x: int):
        self._value = x

def main():
    size = 1000000
    for i in range(size):
        demo_instance = DemoClass(size)
        value = demo_instance.value
        demo_instance.value = i

main()

### 避免数据复制
#### 避免无意义的数据复制
#### 交换值时不使用中间变量
```python
a, b = b, a
```

#### 字符串拼接使用`join`而不是`+`
当使用`a + b`拼接字符串时，由于 Python 中字符串是不可变对象，其会申请一块内存空间，将`a`和`b`分别复制到该新申请的内存空间中。因此，如果要拼接 n 个字符串，会产生 n-1 个中间结果，每产生一个中间结果都需要申请和复制一次内存，严重影响运行效率。而使用`join()`拼接字符串时，会首先计算出需要申请的总的内存空间，然后一次性地申请所需内存，并将每个字符串元素复制到该内存中去。

In [ ]:
# 不推荐写法
import string
from typing import List

def concatString(string_list: List[str]) -> str:
    result = ''
    for str_i in string_list:
        result += str_i
    return result

def main():
    string_list = list(string.ascii_letters * 100)
    for _ in range(10000):
        result = concatString(string_list)

main()

In [ ]:
# 推荐写法
import string
from typing import List

def concatString(string_list: List[str]) -> str:
    return ''.join(string_list)  # 使用 join 而不是 +

def main():
    string_list = list(string.ascii_letters * 100)
    for _ in range(10000):
        result = concatString(string_list)

main()

### 利用 `if` 条件的短路特性
`if` 条件的短路特性是指对`if a and b`这样的语句， 当`a`为`False`时将直接返回，不再计算`b`；对于`if a or b`这样的语句，当`a`为`True`时将直接返回，不再计算`b`。因此， 为了节约运行时间，对于`or`语句，应该将值为`True`可能性比较高的变量写在`or`前，而`and`应该推后。

### 循环优化
- 利用`for`代替`while`；这是由于Python中`for`循环比`while`循环更快；
- 利用隐式`for`循环代替显式`for`循环；例如隐式`sum(range(10000))`；
- 减少内层循环计算；

### 使用`numba.jit`优化
参考另一篇文章：[Python 代码优化技巧（一）](https://dreamhomes.github.io/posts/202005141054.html)

### 使用合适的数据结构
Python 内置的数据结构如`str`, `tuple`, `list`, `set`, `dict`底层都是 C 实现的，速度非常快，自己实现新的数据结构想在性能上达到内置的速度几乎是不可能的。